# Extractor Pipeline Debug Notebook

This notebook compares the original working watershed algorithm with the new ExtractorPipeline implementation to identify and fix differences.

In [ ]:
# Setup and Data Loading
from pathlib import Path
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.stats import entropy as scipy_entropy
from scipy.ndimage import binary_fill_holes, label as ndi_label, distance_transform_edt
from skimage.segmentation import expand_labels, watershed, find_boundaries
from skimage import morphology
import sys

# Project paths
notebook = Path(r"C:\Users\admin\Documents\Pierre lab\projects\Colustrum-ABX\lysozyme stain quantification\code\component development")
project_root = notebook.parent.parent
print(f"Project root: {project_root}")

# Add src to path to import our pipeline
src_path = project_root / "code" / "src"
sys.path.insert(0, str(src_path))
print(f"Added to path: {src_path}")

# Test that we can import our pipeline
try:
    from processing.extractor_pipeline import ExtractorPipeline
    from utils.file_utils import load_as_gray, build_rgb, remove_rectangles
    from utils.image_utils import minmax01
    print("✓ Successfully imported pipeline components")
except ImportError as e:
    print(f"✗ Import failed: {e}")

In [ ]:
# Load test image and prepare helper functions
images_root = project_root / 'lysozyme images'
test_red_path = images_root / "Jej LYZ" / "G2" / "G2EL - 1_RFP.tif"
test_blue_path = images_root / "Jej LYZ" / "G2" / "G2EL - 1_DAPI.tif"

print(f"Red path exists: {test_red_path.exists()}")
print(f"Blue path exists: {test_blue_path.exists()}")

# Original notebook helper functions
def load_as_gray_orig(p: Path):
    """Original notebook version"""
    arr = tifffile.imread(p)
    if arr.ndim == 3:
        if arr.shape[0] <= 4 and arr.shape[0] < arr.shape[-1]:
            arr = np.moveaxis(arr, 0, -1)
        if arr.shape[-1] in (3,4):
            arr = cv2.cvtColor(arr[..., :3].astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            arr = arr[...,0]
    return arr.astype(np.float32)

def build_rgb_orig(red_gray, blue_gray):
    """Original notebook version"""
    def to_u8(x):
        if x.dtype != np.uint8:
            lo, hi = np.nanmin(x), np.nanmax(x)
            if hi > lo:
                x = (x - lo)/(hi-lo)*255.0
            else:
                x = np.zeros_like(x)
            return x.astype(np.uint8)
        return x
    r8 = to_u8(red_gray)
    b8 = to_u8(blue_gray)
    zeros = np.zeros_like(r8)
    return np.stack([r8, zeros, b8], axis=-1)

def minmax01_orig(x, eps=1e-12):
    """Original notebook version"""
    x = x.astype(float, copy=False)
    lo = np.min(x)
    hi = np.max(x)
    return (x - lo) / max(hi - lo, eps)

In [ ]:
# Load and preprocess images - Compare methods
print("=" * 60)
print("COMPARING IMAGE LOADING AND PREPROCESSING")
print("=" * 60)

# Load with original method
r_orig = load_as_gray_orig(test_red_path)
b_orig = load_as_gray_orig(test_blue_path)
print(f"Original method - Red: {r_orig.shape}, range [{r_orig.min():.2f}, {r_orig.max():.2f}]")
print(f"Original method - Blue: {b_orig.shape}, range [{b_orig.min():.2f}, {b_orig.max():.2f}]")

# Load with pipeline method
r_pipeline = load_as_gray(test_red_path)
b_pipeline = load_as_gray(test_blue_path)
print(f"Pipeline method - Red: {r_pipeline.shape}, range [{r_pipeline.min():.2f}, {r_pipeline.max():.2f}]")
print(f"Pipeline method - Blue: {b_pipeline.shape}, range [{b_pipeline.min():.2f}, {b_pipeline.max():.2f}]")

# Check if they're identical
print(f"Red images identical: {np.allclose(r_orig, r_pipeline)}")
print(f"Blue images identical: {np.allclose(b_orig, b_pipeline)}")

# Build RGB with both methods
disp_orig = build_rgb_orig(r_orig, b_orig)
disp_pipeline = build_rgb(r_pipeline, b_pipeline)

print(f"Original RGB: {disp_orig.shape}, dtype {disp_orig.dtype}")
print(f"Pipeline RGB: {disp_pipeline.shape}, dtype {disp_pipeline.dtype}")
print(f"RGB displays identical: {np.allclose(disp_orig, disp_pipeline)}")

# Use original method for the rest (to match notebook exactly)
r = r_orig
b = b_orig
disp = disp_orig

In [ ]:
# ORIGINAL NOTEBOOK ALGORITHM (STEP BY STEP)
print("=" * 60)
print("RUNNING ORIGINAL NOTEBOOK ALGORITHM")
print("=" * 60)

# Apply remove_rectangles
try:
    disp = remove_rectangles(disp)
    print("✓ Rectangle removal applied")
except Exception as e:
    print(f"⚠ Rectangle removal failed: {e}")

# Extract red and blue channels from display
red_orig = disp[...,0].astype(np.float32)
blue_orig = disp[...,2].astype(np.float32)
print(f"Red from display: range [{red_orig.min():.2f}, {red_orig.max():.2f}]")
print(f"Blue from display: range [{blue_orig.min():.2f}, {blue_orig.max():.2f}]")

# Morphological operations
mask_r_dilation_orig = np.maximum(blue_orig, red_orig)
mask_r_erosion_orig = np.minimum(blue_orig, red_orig)
print(f"Dilation mask: range [{mask_r_dilation_orig.min():.2f}, {mask_r_dilation_orig.max():.2f}]")
print(f"Erosion mask: range [{mask_r_erosion_orig.min():.2f}, {mask_r_erosion_orig.max():.2f}]")

# diff_r
diff_r_orig = red_orig > mask_r_erosion_orig
print(f"diff_r raw: {np.sum(diff_r_orig)} pixels")

diff_r_orig = morphology.binary_erosion(diff_r_orig, footprint=np.ones((3,3)))
diff_r_orig = morphology.remove_small_objects(diff_r_orig, min_size=100)
print(f"diff_r final: {np.sum(diff_r_orig)} pixels")

# Secondary mask
abs_diff_orig = np.abs(mask_r_dilation_orig - red_orig)
mask_gt_red_orig = abs_diff_orig > red_orig
print(f"abs_diff range: [{abs_diff_orig.min():.2f}, {abs_diff_orig.max():.2f}]")
print(f"mask_gt_red: {np.sum(mask_gt_red_orig)} pixels")

# Erosion
erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6,6))
mask_u8 = (mask_gt_red_orig.astype(np.uint8)*255)
mask_eroded_u8 = cv2.erode(mask_u8, erosion_kernel, iterations=2)
mask_gt_red_eroded_orig = mask_eroded_u8.astype(bool)
print(f"mask_gt_red_eroded: {np.sum(mask_gt_red_eroded_orig)} pixels")

# Combined labels
combined_labels_orig = np.zeros_like(diff_r_orig, dtype=int)
combined_labels_orig[mask_gt_red_eroded_orig] = 2
combined_labels_orig[diff_r_orig] = 1
unique_combined = np.unique(combined_labels_orig)
counts = [(label, np.sum(combined_labels_orig == label)) for label in unique_combined]
print(f"Combined labels: {counts}")

In [ ]:
# Complete original watershed algorithm
print("=" * 60)
print("COMPLETING ORIGINAL WATERSHED ALGORITHM")
print("=" * 60)

# Expand labels
expanded_labels_orig = expand_labels(combined_labels_orig, distance=100)
unique_expanded = np.unique(expanded_labels_orig)
print(f"Expanded labels: {len(unique_expanded)} unique values")
if len(unique_expanded) <= 20:
    print(f"Expanded values: {unique_expanded}")

# Markers from diff_r
labeled_diff_r_orig, _ = ndi_label(diff_r_orig != 0)
print(f"labeled_diff_r: max {labeled_diff_r_orig.max()} regions")

# Reworked markers
reworked_orig = np.zeros_like(expanded_labels_orig, dtype=np.int32)
reworked_orig[expanded_labels_orig == 2] = 1
mask_copy = (expanded_labels_orig != 2) & (labeled_diff_r_orig != 0)
reworked_orig[mask_copy] = labeled_diff_r_orig[mask_copy] + 1
unique_reworked = np.unique(reworked_orig)
print(f"Reworked markers: {len(unique_reworked)} unique, max: {reworked_orig.max()}")

# Watershed mask
mask_ws_orig = expanded_labels_orig > 0
print(f"Watershed mask: {np.sum(mask_ws_orig)} pixels")

# Elevation
elevation_orig = (
    minmax01_orig(distance_transform_edt(combined_labels_orig == 2))
    - minmax01_orig(distance_transform_edt(combined_labels_orig == 1))
)
print(f"Elevation range: [{elevation_orig.min():.3f}, {elevation_orig.max():.3f}]")

# Final watershed
ws_labels_orig = watershed(elevation_orig, markers=reworked_orig, mask=mask_ws_orig)
unique_ws = np.unique(ws_labels_orig)
print(f"Final watershed: {len(unique_ws)} regions, max label: {ws_labels_orig.max()}")
if len(unique_ws) <= 20:
    print(f"Final labels: {unique_ws}")

# Store original results
orig_results = {
    'ws_labels': ws_labels_orig,
    'expanded_labels': expanded_labels_orig,
    'combined_labels': combined_labels_orig,
    'diff_r': diff_r_orig,
    'mask_gt_red_eroded': mask_gt_red_eroded_orig,
    'disp': disp
}

In [ ]:
# RUN NEW PIPELINE AND COMPARE
print("=" * 60)
print("RUNNING NEW EXTRACTOR PIPELINE")
print("=" * 60)

# Initialize pipeline with debug
extractor = ExtractorPipeline(debug=True)

# Run pipeline (using original loaded images)
ws_labels_pipeline = extractor.extract(r, b)

print("=" * 60)
print("COMPARING RESULTS")
print("=" * 60)

# Compare final results
print(f"Original final labels: {len(np.unique(orig_results['ws_labels']))} regions, max: {orig_results['ws_labels'].max()}")
print(f"Pipeline final labels: {len(np.unique(ws_labels_pipeline))} regions, max: {ws_labels_pipeline.max()}")
print(f"Final results identical: {np.array_equal(orig_results['ws_labels'], ws_labels_pipeline)}")

# Get debug info
debug_info = extractor.get_debug_info()

# Compare intermediate steps
steps_to_compare = [
    ('diff_r', 'diff_r'),
    ('mask_gt_red_eroded', 'mask_gt_red_eroded'), 
    ('combined_labels', 'combined_labels'),
    ('expanded_labels', 'expanded_labels'),
    ('ws_labels', 'ws_labels')
]

for orig_key, pipeline_key in steps_to_compare:
    if orig_key in orig_results and pipeline_key in debug_info:
        orig_data = orig_results[orig_key]
        pipeline_data = debug_info[pipeline_key]
        identical = np.array_equal(orig_data, pipeline_data)
        print(f"{orig_key:20} identical: {identical}")
        if not identical:
            print(f"  Original unique: {np.unique(orig_data)}")
            print(f"  Pipeline unique: {np.unique(pipeline_data)}")
            if orig_data.dtype == bool:
                print(f"  Original sum: {np.sum(orig_data)}")
                print(f"  Pipeline sum: {np.sum(pipeline_data)}")
    else:
        print(f"{orig_key:20} - Missing data for comparison")

In [ ]:
# DETAILED CHANNEL COMPARISON
print("=" * 60)
print("DETAILED CHANNEL ANALYSIS")
print("=" * 60)

# Compare the critical step: channel extraction from RGB
red_orig_extracted = red_orig  # From original: disp[...,0].astype(np.float32)
blue_orig_extracted = blue_orig  # From original: disp[...,2].astype(np.float32)

if 'red_from_disp' in debug_info and 'blue_from_disp' in debug_info:
    red_pipeline_extracted = debug_info['red_from_disp']
    blue_pipeline_extracted = debug_info['blue_from_disp']
    
    print(f"Red channel comparison:")
    print(f"  Original range: [{red_orig_extracted.min():.2f}, {red_orig_extracted.max():.2f}]")
    print(f"  Pipeline range: [{red_pipeline_extracted.min():.2f}, {red_pipeline_extracted.max():.2f}]")
    print(f"  Identical: {np.allclose(red_orig_extracted, red_pipeline_extracted)}")
    
    print(f"Blue channel comparison:")
    print(f"  Original range: [{blue_orig_extracted.min():.2f}, {blue_orig_extracted.max():.2f}]")
    print(f"  Pipeline range: [{blue_pipeline_extracted.min():.2f}, {blue_pipeline_extracted.max():.2f}]")
    print(f"  Identical: {np.allclose(blue_orig_extracted, blue_pipeline_extracted)}")
    
    # Check if the RGB displays are identical
    if 'disp_rgb' in debug_info:
        disp_pipeline = debug_info['disp_rgb']
        print(f"RGB display identical: {np.array_equal(disp, disp_pipeline)}")
        if not np.array_equal(disp, disp_pipeline):
            print(f"  Original RGB range: [{disp.min()}, {disp.max()}]")
            print(f"  Pipeline RGB range: [{disp_pipeline.min()}, {disp_pipeline.max()}]")
else:
    print("⚠ Debug info missing channel data")

# Check morphological operations
if 'mask_r_dilation' in debug_info and 'mask_r_erosion' in debug_info:
    print(f"\nMorphological operations:")
    print(f"  Dilation identical: {np.allclose(mask_r_dilation_orig, debug_info['mask_r_dilation'])}")
    print(f"  Erosion identical: {np.allclose(mask_r_erosion_orig, debug_info['mask_r_erosion'])}")

In [ ]:
# VISUALIZATION OF RESULTS
print("=" * 60)
print("CREATING COMPARISON VISUALIZATIONS")
print("=" * 60)

# Create side-by-side comparison
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Row 1: Original algorithm results
row = 0
axes[row, 0].imshow(orig_results['diff_r'], cmap='gray')
axes[row, 0].set_title('Original: diff_r')
axes[row, 0].axis('off')

axes[row, 1].imshow(orig_results['combined_labels'], cmap='tab10')
axes[row, 1].set_title('Original: combined_labels')
axes[row, 1].axis('off')

axes[row, 2].imshow(orig_results['expanded_labels'], cmap='tab10')
axes[row, 2].set_title('Original: expanded_labels')
axes[row, 2].axis('off')

axes[row, 3].imshow(orig_results['ws_labels'], cmap='tab20')
axes[row, 3].set_title('Original: watershed')
axes[row, 3].axis('off')

# Row 2: Pipeline results
row = 1
if 'diff_r' in debug_info:
    axes[row, 0].imshow(debug_info['diff_r'], cmap='gray')
    axes[row, 0].set_title('Pipeline: diff_r')
    axes[row, 0].axis('off')

if 'combined_labels' in debug_info:
    axes[row, 1].imshow(debug_info['combined_labels'], cmap='tab10')
    axes[row, 1].set_title('Pipeline: combined_labels')
    axes[row, 1].axis('off')

if 'expanded_labels' in debug_info:
    axes[row, 2].imshow(debug_info['expanded_labels'], cmap='tab10')
    axes[row, 2].set_title('Pipeline: expanded_labels')
    axes[row, 2].axis('off')

axes[row, 3].imshow(ws_labels_pipeline, cmap='tab20')
axes[row, 3].set_title('Pipeline: watershed')
axes[row, 3].axis('off')

plt.tight_layout()
plt.show()

# Overlay comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original overlay
boundaries_orig = find_boundaries(orig_results['ws_labels'], mode='inner')
overlay_orig = orig_results['disp'].copy()
overlay_orig[boundaries_orig] = [255, 0, 0]

axes[0].imshow(overlay_orig)
axes[0].set_title(f'Original Algorithm\n{len(np.unique(orig_results["ws_labels"]))} regions')
axes[0].axis('off')

# Pipeline overlay
boundaries_pipeline = find_boundaries(ws_labels_pipeline, mode='inner')
overlay_pipeline = orig_results['disp'].copy()
overlay_pipeline[boundaries_pipeline] = [0, 255, 0]

axes[1].imshow(overlay_pipeline)
axes[1].set_title(f'Pipeline Algorithm\n{len(np.unique(ws_labels_pipeline))} regions')
axes[1].axis('off')

# Combined overlay (red=original, green=pipeline)
overlay_combined = orig_results['disp'].copy()
overlay_combined[boundaries_orig] = [255, 0, 0]    # Red for original
overlay_combined[boundaries_pipeline] = [0, 255, 0]  # Green for pipeline

axes[2].imshow(overlay_combined)
axes[2].set_title('Combined Overlay\nRed=Original, Green=Pipeline')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# DIAGNOSTIC ANALYSIS
print("=" * 60)
print("DIAGNOSTIC ANALYSIS AND RECOMMENDATIONS")
print("=" * 60)

# Analyze where the differences occur
if not np.array_equal(orig_results['ws_labels'], ws_labels_pipeline):
    print("❌ ALGORITHMS PRODUCE DIFFERENT RESULTS")
    
    # Find first point of divergence
    divergence_found = False
    
    # Check each step
    check_steps = [
        ('red channels', red_orig_extracted, debug_info.get('red_from_disp')),
        ('blue channels', blue_orig_extracted, debug_info.get('blue_from_disp')),
        ('diff_r', orig_results['diff_r'], debug_info.get('diff_r')),
        ('mask_gt_red_eroded', orig_results['mask_gt_red_eroded'], debug_info.get('mask_gt_red_eroded')),
        ('combined_labels', orig_results['combined_labels'], debug_info.get('combined_labels')),
        ('expanded_labels', orig_results['expanded_labels'], debug_info.get('expanded_labels')),
        ('final_labels', orig_results['ws_labels'], ws_labels_pipeline)
    ]
    
    for step_name, orig_data, pipeline_data in check_steps:
        if orig_data is not None and pipeline_data is not None:
            if isinstance(orig_data, np.ndarray) and isinstance(pipeline_data, np.ndarray):
                if not np.array_equal(orig_data, pipeline_data):
                    print(f"🔍 DIVERGENCE FOUND at: {step_name}")
                    print(f"   Original shape: {orig_data.shape}, dtype: {orig_data.dtype}")
                    print(f"   Pipeline shape: {pipeline_data.shape}, dtype: {pipeline_data.dtype}")
                    if orig_data.dtype == bool:
                        print(f"   Original True pixels: {np.sum(orig_data)}")
                        print(f"   Pipeline True pixels: {np.sum(pipeline_data)}")
                    else:
                        print(f"   Original unique values: {len(np.unique(orig_data))}")
                        print(f"   Pipeline unique values: {len(np.unique(pipeline_data))}")
                        print(f"   Original range: [{orig_data.min()}, {orig_data.max()}]")
                        print(f"   Pipeline range: [{pipeline_data.min()}, {pipeline_data.max()}]")
                    divergence_found = True
                    break
                else:
                    print(f"✓ {step_name}: IDENTICAL")
            else:
                print(f"⚠ {step_name}: Cannot compare (type mismatch)")
        else:
            print(f"⚠ {step_name}: Missing data")
    
    if not divergence_found:
        print("🤔 No clear divergence point found - may be floating point precision issue")

else:
    print("✅ ALGORITHMS PRODUCE IDENTICAL RESULTS!")

# Summary statistics
print(f"\nSUMMARY:")
print(f"Original algorithm found: {len(np.unique(orig_results['ws_labels']))} regions")
print(f"Pipeline algorithm found: {len(np.unique(ws_labels_pipeline))} regions")
print(f"Test image: {test_red_path.name}")